# Libraries

In [1]:
import numpy as np
import pandas as pd
import torch

from bib_sensing.TorchSOM.TorchSOM.core import TorchSOM # To modify
from bib_sensing.TorchSOM.TorchSOM.plotting import SOMVisualizer, VisualizationConfig # To modify

In [2]:
random_seed = 42
torch.manual_seed(random_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing 

In [3]:
iris_df = pd.read_csv(
    filepath_or_buffer="../data/iris.csv",
)


In [4]:
iris_df['Species'] = iris_df['Species'].map({
    'Iris-setosa': 0, 
    'Iris-versicolor': 1, 
    'Iris-virginica': 2}
)

In [5]:
iris_df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
feature_names = iris_df.columns.to_list()[:4]
feature_names

['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']

In [7]:
iris_df.shape

(150, 5)

In [8]:
"""
1. Create a tensor from the iris df and separate the features and the target
2. Randomly shuffle the data
3. Split the data into training and testing sets
"""
iris_torch = torch.tensor(iris_df.to_numpy(dtype=np.float32))
all_features, all_targets = iris_torch[:, :4], iris_torch[:, 4].long()


shuffled_indices = torch.randperm(len(all_features))
all_features, all_targets = all_features[shuffled_indices], all_targets[shuffled_indices]

train_ratio = 0.8
train_count = int(train_ratio * len(all_features))
train_features, train_targets = all_features[:train_count], all_targets[:train_count]
test_features, test_targets = all_features[train_count:], all_targets[train_count:]

print(train_features.shape, test_features.shape)
print(train_targets.shape, test_targets.shape)

torch.Size([120, 4]) torch.Size([30, 4])
torch.Size([120]) torch.Size([30])


# TorchSOM

In [38]:
som = TorchSOM(
    x=10,
    y=10,
    sigma=2.5,
    learning_rate=0.95,
    neighborhood_order=3,
    epochs=200,
    batch_size=16,
    topology="rectangular",
    distance_function="euclidean",
    neighborhood_function="gaussian",
    num_features=all_features.shape[1],
    lr_decay_function="asymptotic_decay",
    sigma_decay_function="asymptotic_decay",
    initialization_mode="pca",
    device=device,
    random_seed=random_seed,
) 

In [39]:
som.initialize_weights(
    data=train_features,
)

In [40]:
QE, TE = som.fit(
    data=train_features
)

Training SOM: 100%|██████████| 200/200 [00:01<00:00, 100.29epoch/s]


In [41]:
visualizer = SOMVisualizer(som=som, config=None)
save_path = "results/iris" # Set to None if you want a direct plot

In [42]:
visualizer.plot_training_errors(
    quantization_errors=QE, 
    topographic_errors=TE, 
    save_path=save_path
)

In [43]:
visualizer.plot_distance_map(save_path=save_path)

In [44]:
visualizer.plot_hit_map(
    data=train_features,
    save_path=save_path
)

In [45]:
visualizer.plot_component_planes(
    component_names=feature_names,
    save_path=save_path
)